In [1]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

2023-07-01 01:17:56.253515: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-01 01:17:56.419237: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-01 01:17:56.420585: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-01 01:17:57.836625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_years = 1

In [3]:
end_time = datetime.now()
start_time = datetime(
    end_time.year - n_years,
    end_time.month,
    end_time.day
)

In [4]:
def get_train_data(stock_name, column, start, end, scaler, fit=False):
    print(f'downloading {stock_name}')
    df = pdr.get_data_yahoo(
        stock_name,
        start,
        end
    )
    data = df.filter([column])
    dataset = data.values
    
    scaled_data = scaler.fit_transform(dataset) if fit else scaler.transform(dataset)

    X = []
    y = []

    for i in range(60, len(scaled_data)):
        X.append(scaled_data[i-60:i, 0])
        y.append(scaled_data[i, 0])
    
    return X, y

In [5]:
def get_aggregated_train_data(train_stocks, column, start_time, end_time):
    scaler = MinMaxScaler(feature_range=(0,1))

    X_train = []
    y_train = []

    for i, stock in enumerate(train_stocks):
        try:
            X, y = get_train_data(stock, column, start_time, end_time, scaler, fit=(i==0))
            X_train += X
            y_train += y
        except:
            continue
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    with open(f'scaler_{column}.pickle', 'wb') as pkl:
        pickle.dump(scaler, pkl)

    return X_train, y_train

In [6]:
def compile_save_model(column, X_train, y_train):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)),
        LSTM(64, return_sequences=True),
        LSTM(64, return_sequences=False),
        Dense(25),
        Dense(25),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, batch_size=1, epochs=1)
    
    model.save(f'model_{column}.h5', save_format='h5')

## Training All Models

In [7]:
train_stocks = [
    'AAPL',
    'BIVI',
    'GOOG',
    'NVMI',
    'CHRD',
    'ASMB',
    'GOGO',
    'CAFG',
    'BVXV',
    'AIBBU',
    'LGVCW'
]
columns = [
    'Open',
    'High',
    'Low',
    'Close',
    'Adj Close',
    'Volume'
]

In [8]:
for col in columns:
    X_train, y_train = get_aggregated_train_data(train_stocks, col, start_time, end_time)
    compile_save_model(col, X_train, y_train)

downloading AAPL


[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[*********************100%***********************]  1 of 1 completed


2023-07-01 01:18:17.787146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 01:18:17.789575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 01:18:17.791030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1718/1718 [==============================] - 85s 46ms/step - loss: 0.0121
downloading AAPL
[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[*********************100%*********************

2023-07-01 01:20:29.064070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 01:20:29.068274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 01:20:29.070375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

1718/1718 [==============================] - 84s 46ms/step - loss: 0.0136
downloading AAPL
[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[*********************100%*********************

2023-07-01 01:25:48.616167: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 01:25:48.617852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 01:25:48.621198: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1718/1718 [==============================] - 86s 47ms/step - loss: 0.0115
downloading AAPL
[*********************100%***********************]  1 of 1 completed
downloading BIVI
[*********************100%***********************]  1 of 1 completed
downloading GOOG
[*********************100%***********************]  1 of 1 completed
downloading NVMI
[*********************100%***********************]  1 of 1 completed
downloading CHRD
[*********************100%***********************]  1 of 1 completed
downloading ASMB
[*********************100%***********************]  1 of 1 completed
downloading GOGO
[*********************100%***********************]  1 of 1 completed
downloading CAFG
[*********************100%***********************]  1 of 1 completed
downloading BVXV
[*********************100%***********************]  1 of 1 completed
downloading AIBBU
[*********************100%***********************]  1 of 1 completed
downloading LGVCW
[*********************100%*********************

2023-07-01 01:27:33.071657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-01 01:27:33.075025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-01 01:27:33.076740: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1718/1718 [==============================] - 85s 47ms/step - loss: 0.0064
